<a href="https://colab.research.google.com/github/GabrieleSocrate/Text-Mining-/blob/main/Dataset8K.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

# 1. Configurazione Aziende (Microsoft, Apple, Nvidia)
aziende_cik = {
    'Microsoft': '0000789019',
    'Apple': '0000320193',
    'Nvidia': '0001045810'
}

# IMPORTANTE: Inserisci una tua email reale, altrimenti la SEC bloccherà le richieste
headers = {'User-Agent': 'TuoNome TuaEmail@esempio.com'}

# Lista per raccogliere i dataframe di ogni azienda
lista_df = []

print("Recupero degli indici SEC per le aziende...")

# 2. Ciclo per scaricare i metadati di ogni singola azienda
for nome_azienda, cik in aziende_cik.items():
    url_submissions = f"https://data.sec.gov/submissions/CIK{cik.zfill(10)}.json"

    response = requests.get(url_submissions, headers=headers)
    if response.status_code == 200:
        data = response.json()
        filings = data['filings']['recent']

        # Creazione DataFrame temporaneo
        df_temp = pd.DataFrame({
            'Azienda': nome_azienda,
            'CIK': cik,
            'Data': filings['filingDate'],
            'Ora': filings['acceptanceDateTime'],
            'Tipo': filings['form'],
            'Accession': filings['accessionNumber'],
            'PrimaryDoc': filings['primaryDocument']
        })

        # 3. Filtriamo 8-K, 10-K, 10-Q e teniamo SOLO I PRIMI 5 per questa azienda
        df_filtrato = df_temp[df_temp['Tipo'].isin(['8-K', '10-K', '10-Q'])].head(5).copy()
        lista_df.append(df_filtrato)
        print(f"- Trovati i documenti per {nome_azienda}")
    else:
        print(f"Errore {response.status_code} per {nome_azienda}")

    # Pausa di cortesia tra una richiesta JSON e l'altra
    time.sleep(0.2)

# Uniamo tutti i risultati in un unico grande DataFrame (15 righe in totale)
df_articoli = pd.concat(lista_df, ignore_index=True)

# 4. Funzione per scaricare e pulire il testo
def download_sec_text(cik, accession, primary_doc):
    time.sleep(0.2) # Pausa obbligatoria per non sovraccaricare il server

    accession_clean = accession.replace('-', '')
    cik_short = str(int(cik))
    url_doc = f"https://www.sec.gov/Archives/edgar/data/{cik_short}/{accession_clean}/{primary_doc}"

    try:
        res = requests.get(url_doc, headers=headers)
        if res.status_code == 200:
            soup = BeautifulSoup(res.content, 'html.parser')

            # Pulizia HTML e metadati XBRL
            for tag in soup(["script", "style"]):
                tag.decompose()
            for ix_tag in soup.find_all(['ix:header', 'ix:hidden']):
                ix_tag.decompose()
            for hidden_tag in soup.find_all(style=lambda value: value and 'display:none' in value.replace(' ', '')):
                hidden_tag.decompose()

            return soup.get_text(separator=' ', strip=True)
        else:
            return f"Errore HTTP {res.status_code}"
    except Exception as e:
        return f"Errore nel download: {e}"

print("\nDownload e pulizia in corso dei 15 documenti testuali... (potrebbe volerci un po')")

# 5. Applichiamo la funzione, passando il CIK specifico per ogni riga
df_articoli['Articolo'] = df_articoli.apply(
    lambda row: download_sec_text(row['CIK'], row['Accession'], row['PrimaryDoc']), axis=1
)

# 6. Pulizia finale della colonna Ora e rimozione colonne tecniche inutili
df_articoli['Ora'] = pd.to_datetime(df_articoli['Ora']).dt.time
df_finale = df_articoli[['Azienda', 'Tipo', 'Data', 'Ora', 'Articolo']]

# Mostriamo il risultato finale
print(df_finale)

# Aggiungi questo alla fine del tuo codice Fase 1
df_finale.to_csv("fase1_testi_sec.csv", index=False)

Recupero degli indici SEC per le aziende...
- Trovati i documenti per Microsoft
- Trovati i documenti per Apple
- Trovati i documenti per Nvidia

Download e pulizia in corso dei 15 documenti testuali... (potrebbe volerci un po')
      Azienda  Tipo        Data       Ora  \
0   Microsoft  10-Q  2026-01-28  21:07:34   
1   Microsoft   8-K  2026-01-28  21:04:38   
2   Microsoft   8-K  2025-12-08  21:00:35   
3   Microsoft  10-Q  2025-10-29  20:10:48   
4   Microsoft   8-K  2025-10-29  20:07:42   
5       Apple  10-Q  2026-01-30  11:01:32   
6       Apple   8-K  2026-01-29  21:30:33   
7       Apple   8-K  2026-01-02  21:30:52   
8       Apple   8-K  2025-12-05  21:31:42   
9       Apple  10-K  2025-10-31  10:01:26   
10     Nvidia   8-K  2026-01-23  22:01:33   
11     Nvidia  10-Q  2025-11-19  21:36:17   
12     Nvidia   8-K  2025-11-19  21:20:44   
13     Nvidia  10-Q  2025-08-27  20:52:07   
14     Nvidia   8-K  2025-08-27  20:22:14   

                                             Artic

In [ ]:
# 1. Impostazioni: scegli quale articolo e quante parole vuoi vedere
indice_articolo = 0  # 0 è il primo articolo (Microsoft), 1 è il secondo, ecc.
numero_parole = 1000  # Modifica questo numero con le parole che desideri

# 2. Estraiamo il testo completo della riga scelta
testo_completo = df_finale['Articolo'].iloc[indice_articolo]

# 3. Dividiamo il testo in parole, prendiamo le prime 'X' e le riuniamo con uno spazio
parole_estratte = ' '.join(testo_completo.split()[:numero_parole])

# 4. Stampiamo il risultato
print(f"\n--- Prime {numero_parole} parole dell'articolo {indice_articolo} ({df_finale['Azienda'].iloc[indice_articolo]} - {df_finale['Tipo'].iloc[indice_articolo]}) ---")
print(parole_estratte)
print("... [CONTINUA]")


--- Prime 1000 parole dell'articolo 0 (Microsoft - 10-Q) ---
10-Q UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549 FORM 10- Q ☒ QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the Quarterly Period Ended December 31, 2025 OR ☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the Transition Period From to Commission File Number 001-37845 MICROSOFT CORPORATION Washington 91-1144442 (STATE OF INCORPORATION) (I.R.S. ID) ONE MICROSOFT WAY , REDMOND , Washington 98052-6399 ( 425 ) 882-8080 www.microsoft.com/investor Securities registered pursuant to Section 12(b) of the Act: Title of each class Trading Symbol Name of exchange on which registered Common stock, $0.00000625 par value per share MSFT Nasdaq 3.125% Notes due 2028 MSFT Nasdaq 2.625% Notes due 2033 MSFT Nasdaq Indicate by check mark whether the registrant (1) has filed all reports required to be filed by Section 13 or 15(d)

In [ ]:
import requests
import pandas as pd
import time
import os
from datetime import timedelta

# --- CONFIGURAZIONI FASE 2 ---
POLYGON_API_KEY = "IOfA_Np3cXfnuFJE_fzI3gMmoxdsT39p" # Inserisci la tua chiave API qui
FILE_INPUT = "fase1_testi_sec.csv"         # Il file generato dal tuo codice Fase 1
FILE_OUTPUT = "dataset_finale_completo.csv"

# Il tuo codice Fase 1 ha salvato il nome dell'azienda, ma a Polygon serve il Ticker.
# Creiamo un dizionario per fare la conversione inversa:
mappa_ticker = {
    'Microsoft': 'MSFT',
    'Apple': 'AAPL',
    'Nvidia': 'NVDA'
}

# --- FUNZIONE POLYGON (LENTA MA PRECISA) ---
def ottieni_prezzi_polygon(ticker, data_str, ora_str):
    time.sleep(13) # Rispetto rigoroso del limite gratuito (5 richieste al minuto)
    try:
        # Ricostruiamo la data e l'ora esatta in UTC (come fornite dalla SEC)
        dt_utc = pd.to_datetime(f"{data_str} {ora_str}").tz_localize('UTC')
        # Convertiamo nel fuso orario di Wall Street
        dt_pub = dt_utc.tz_convert('America/New_York')

        start_date = dt_pub.strftime('%Y-%m-%d')
        end_date = (dt_pub + timedelta(days=4)).strftime('%Y-%m-%d') # Finestra per superare i weekend

        url_poly = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/minute/{start_date}/{end_date}?adjusted=true&sort=asc&apiKey={POLYGON_API_KEY}"
        res_poly = requests.get(url_poly)

        if res_poly.status_code != 200: return "Err API", None, None

        data_poly = res_poly.json()
        if 'results' not in data_poly or len(data_poly['results']) == 0: return "No Dati", None, None

        df_prezzi = pd.DataFrame(data_poly['results'])
        df_prezzi['datetime'] = pd.to_datetime(df_prezzi['t'], unit='ms', utc=True).dt.tz_convert('America/New_York')

        # Filtriamo per trovare la prima apertura di mercato disponibile
        df_successivo = df_prezzi[df_prezzi['datetime'] >= dt_pub]
        if df_successivo.empty: return "Mercato Chiuso", None, None

        t0_time = df_successivo['datetime'].iloc[0]
        prezzo_t0 = float(df_successivo['c'].iloc[0])

        # Calcoliamo la mezz'ora dopo
        t30_target = t0_time + timedelta(minutes=30)
        df_t30 = df_successivo[df_successivo['datetime'] >= t30_target]
        prezzo_t30 = float(df_successivo['c'].iloc[-1]) if df_t30.empty else float(df_t30['c'].iloc[0])

        delta_perc = ((prezzo_t30 - prezzo_t0) / prezzo_t0) * 100

        # Restituiamo anche l'orario effettivo di mercato in cui è stata calcolata l'apertura
        return round(prezzo_t0, 2), round(prezzo_t30, 2), round(delta_perc, 2), t0_time.strftime('%Y-%m-%d %H:%M:%S ET')
    except Exception as e:
        return f"Err: {e}", None, None, None

# --- ESECUZIONE PRINCIPALE ---
if not os.path.exists(FILE_INPUT):
    print(f"Errore: Non trovo il file {FILE_INPUT}. Esegui prima il codice Fase 1 e assicurati di aver aggiunto df_finale.to_csv(...) alla fine.")
    exit()

df_testi = pd.read_csv(FILE_INPUT)

# GESTIONE RIPRESA (RESUME)
# Poiché non abbiamo l'ID 'Accession', usiamo la combinazione univoca di Azienda+Data+Ora per capire se un articolo è già stato fatto.
documenti_fatti = set()
if os.path.exists(FILE_OUTPUT):
    df_esistente = pd.read_csv(FILE_OUTPUT)
    # Creiamo una chiave fittizia per identificare la riga
    documenti_fatti = set(df_esistente['Azienda'] + df_esistente['Data_SEC'] + df_esistente['Ora_SEC'])
    print(f"-> Trovati {len(documenti_fatti)} articoli già processati. Riprendo da dove mi ero fermato...")

print(f"Inizio calcolo prezzi. (Premi Ctrl+C nel terminale per fermare il processo e salvare)\n")

for index, row in df_testi.iterrows():
    chiave_univoca = row['Azienda'] + str(row['Data']) + str(row['Ora'])

    if chiave_univoca in documenti_fatti:
        continue # Salta questo articolo se è già nel file di output

    try:
        ticker = mappa_ticker[row['Azienda']]
        print(f"Elaboro prezzi per {row['Azienda']} del {row['Data']} alle {row['Ora']}...")

        # Interroga Polygon
        p_t0, p_t30, delta, ora_mercato = ottieni_prezzi_polygon(ticker, str(row['Data']), str(row['Ora']))

        # Crea la riga completa con i dati testuali inglesi e i nuovi dati finanziari
        nuova_riga = pd.DataFrame({
            'Azienda': [row['Azienda']],
            'Ticker': [ticker],
            'Tipo_Documento': [row['Tipo']],
            'Data_SEC': [row['Data']],
            'Ora_SEC': [row['Ora']],
            'Ora_Effettiva_Mercato': [ora_mercato],
            'Prezzo_T0': [p_t0],
            'Prezzo_T30': [p_t30],
            'Delta_%': [delta],
            'Articolo_EN': [row['Articolo']] # Il dataset testuale rimane intatto
        })

        # Salva in APPEND
        include_header = not os.path.exists(FILE_OUTPUT)
        nuova_riga.to_csv(FILE_OUTPUT, mode='a', index=False, header=include_header)

    except KeyboardInterrupt:
        print("\n[!] Interruzione manuale. Il processo si è fermato in sicurezza. Dati salvati.")
        break
    except Exception as e:
        print(f"Errore strano: {e}")
        continue

print("\nElaborazione terminata! Controlla il file:", FILE_OUTPUT)

Inizio calcolo prezzi. (Premi Ctrl+C nel terminale per fermare il processo e salvare)

Elaboro prezzi per Microsoft del 2026-01-28 alle 21:07:34...
Elaboro prezzi per Microsoft del 2026-01-28 alle 21:04:38...
Elaboro prezzi per Microsoft del 2025-12-08 alle 21:00:35...
Elaboro prezzi per Microsoft del 2025-10-29 alle 20:10:48...
Elaboro prezzi per Microsoft del 2025-10-29 alle 20:07:42...
Elaboro prezzi per Apple del 2026-01-30 alle 11:01:32...
Elaboro prezzi per Apple del 2026-01-29 alle 21:30:33...
Elaboro prezzi per Apple del 2026-01-02 alle 21:30:52...
Elaboro prezzi per Apple del 2025-12-05 alle 21:31:42...
Elaboro prezzi per Apple del 2025-10-31 alle 10:01:26...
Elaboro prezzi per Nvidia del 2026-01-23 alle 22:01:33...
Elaboro prezzi per Nvidia del 2025-11-19 alle 21:36:17...
Elaboro prezzi per Nvidia del 2025-11-19 alle 21:20:44...
Elaboro prezzi per Nvidia del 2025-08-27 alle 20:52:07...
Elaboro prezzi per Nvidia del 2025-08-27 alle 20:22:14...

Elaborazione terminata! Controll

In [ ]:
import pandas as pd

# 1. Carichiamo il file appena creato
df_verifica = pd.read_csv("dataset_finale_completo.csv")

# 2. Stampiamo un riepilogo generale (quante righe, quante colonne)
print("=== INFO GENERALI DATASET ===")
print(df_verifica.info())
print("-" * 50)

# 3. Controlliamo se i calcoli finanziari hanno senso
# Mostriamo solo le colonne chiave per le prime 5 righe
print("\n=== VERIFICA PREZZI E DELTA (Prime 5 righe) ===")
colonne_finanza = ['Azienda', 'Tipo_Documento', 'Data_SEC', 'Prezzo_T0', 'Prezzo_T30', 'Delta_%']
print(df_verifica[colonne_finanza].head())
print("-" * 50)

# 4. Controlliamo la purezza del testo per il text mining
# Prendiamo le prime 300 parole del primo articolo per assicurarci che sia in inglese e senza codice HTML
print("\n=== VERIFICA TESTO (Estratto primo documento) ===")
primo_articolo = str(df_verifica['Articolo_EN'].iloc[0])
estratto_parole = ' '.join(primo_articolo.split()[:50]) # Prende le prime 50 parole
print(estratto_parole + " [...]")


=== INFO GENERALI DATASET ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Azienda                15 non-null     object 
 1   Ticker                 15 non-null     object 
 2   Tipo_Documento         15 non-null     object 
 3   Data_SEC               15 non-null     object 
 4   Ora_SEC                15 non-null     object 
 5   Ora_Effettiva_Mercato  15 non-null     object 
 6   Prezzo_T0              15 non-null     float64
 7   Prezzo_T30             15 non-null     float64
 8   Delta_%                15 non-null     float64
 9   Articolo_EN            15 non-null     object 
dtypes: float64(3), object(7)
memory usage: 1.3+ KB
None
--------------------------------------------------

=== VERIFICA PREZZI E DELTA (Prime 5 righe) ===
     Azienda Tipo_Documento    Data_SEC  Prezzo_T0  Prezzo_T30  Delta_%
0  Microsoft  

In [ ]:
import requests
import pandas as pd
import time
import os
from datetime import timedelta

# --- CONFIGURAZIONI FASE 2 ---
POLYGON_API_KEY = "IOfA_Np3cXfnuFJE_fzI3gMmoxdsT39p" # Cambia la chiave, la precedente era esposta!
FILE_INPUT = "fase1_testi_sec.csv"
# CAMBIATO NOME FILE: Avendo nuove colonne, creiamo un dataset pulito
FILE_OUTPUT = "dataset_finale_quant.csv"

mappa_ticker = {
    'Microsoft': 'MSFT',
    'Apple': 'AAPL',
    'Nvidia': 'NVDA'
}

# --- FUNZIONE POLYGON AVANZATA (STOCK + INDEX + VOLUME) ---
def ottieni_prezzi_polygon(ticker, data_str, ora_str):
    # Pausa di 26 secondi: facciamo 2 chiamate ad articolo (Azione + Indice)
    # Limite gratuito: 5 chiamate al minuto. Così siamo al sicuro.
    time.sleep(26)

    try:
        dt_utc = pd.to_datetime(f"{data_str} {ora_str}").tz_localize('UTC')
        dt_pub = dt_utc.tz_convert('America/New_York')

        start_date = dt_pub.strftime('%Y-%m-%d')
        end_date = (dt_pub + timedelta(days=4)).strftime('%Y-%m-%d')

        # --- 1. CHIAMATA AZIENDA (es. MSFT) ---
        url_stock = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/minute/{start_date}/{end_date}?adjusted=true&sort=asc&apiKey={POLYGON_API_KEY}"
        res_stock = requests.get(url_stock)

        if res_stock.status_code != 200: return "Err API Stock", None, None, None, None, None, None

        data_stock = res_stock.json()
        if 'results' not in data_stock or len(data_stock['results']) == 0: return "No Dati Stock", None, None, None, None, None, None

        df_stock = pd.DataFrame(data_stock['results'])
        df_stock['datetime'] = pd.to_datetime(df_stock['t'], unit='ms', utc=True).dt.tz_convert('America/New_York')

        df_succ_stock = df_stock[df_stock['datetime'] >= dt_pub]
        if df_succ_stock.empty: return "Mercato Chiuso", None, None, None, None, None, None

        t0_time = df_succ_stock['datetime'].iloc[0]
        prezzo_t0 = float(df_succ_stock['c'].iloc[0])
        volume_t0 = int(df_succ_stock['v'].iloc[0]) # NOVITÀ: Estrazione del Volume

        t30_target = t0_time + timedelta(minutes=30)
        df_t30 = df_succ_stock[df_succ_stock['datetime'] >= t30_target]
        prezzo_t30 = float(df_succ_stock['c'].iloc[-1]) if df_t30.empty else float(df_t30['c'].iloc[0])

        delta_stock = ((prezzo_t30 - prezzo_t0) / prezzo_t0) * 100

        # --- 2. CHIAMATA BENCHMARK (QQQ - Nasdaq) ---
        url_index = f"https://api.polygon.io/v2/aggs/ticker/QQQ/range/1/minute/{start_date}/{end_date}?adjusted=true&sort=asc&apiKey={POLYGON_API_KEY}"
        res_index = requests.get(url_index)

        if res_index.status_code != 200: return round(prezzo_t0, 2), round(prezzo_t30, 2), round(delta_stock, 2), "Err Index", "Err Index", volume_t0, t0_time.strftime('%Y-%m-%d %H:%M:%S ET')

        data_index = res_index.json()
        if 'results' not in data_index or len(data_index['results']) == 0:
            return round(prezzo_t0, 2), round(prezzo_t30, 2), round(delta_stock, 2), "No Dati Index", "No Dati Index", volume_t0, t0_time.strftime('%Y-%m-%d %H:%M:%S ET')

        df_index = pd.DataFrame(data_index['results'])
        df_index['datetime'] = pd.to_datetime(df_index['t'], unit='ms', utc=True).dt.tz_convert('America/New_York')

        # Cerchiamo l'indice agli stessi minuti esatti dell'azione
        df_idx_t0 = df_index[df_index['datetime'] == t0_time]
        df_idx_t30 = df_index[df_index['datetime'] == (df_t30['datetime'].iloc[0] if not df_t30.empty else df_succ_stock['datetime'].iloc[-1])]

        if df_idx_t0.empty or df_idx_t30.empty:
            return round(prezzo_t0, 2), round(prezzo_t30, 2), round(delta_stock, 2), "Manca IDX", "Manca IDX", volume_t0, t0_time.strftime('%Y-%m-%d %H:%M:%S ET')

        idx_p0 = float(df_idx_t0['c'].iloc[0])
        idx_p30 = float(df_idx_t30['c'].iloc[0])
        delta_index = ((idx_p30 - idx_p0) / idx_p0) * 100

        # --- 3. CALCOLO ABNORMAL RETURN ---
        abnormal_return = delta_stock - delta_index

        return round(prezzo_t0, 2), round(prezzo_t30, 2), round(delta_stock, 2), round(delta_index, 2), round(abnormal_return, 2), volume_t0, t0_time.strftime('%Y-%m-%d %H:%M:%S ET')

    except Exception as e:
        return f"Err: {e}", None, None, None, None, None, None

# --- ESECUZIONE PRINCIPALE ---
if not os.path.exists(FILE_INPUT):
    print(f"Errore: Non trovo il file {FILE_INPUT}. Esegui prima il codice Fase 1.")
    exit()

df_testi = pd.read_csv(FILE_INPUT)

documenti_fatti = set()
if os.path.exists(FILE_OUTPUT):
    df_esistente = pd.read_csv(FILE_OUTPUT)
    documenti_fatti = set(df_esistente['Azienda'] + df_esistente['Data_SEC'] + df_esistente['Ora_SEC'])
    print(f"-> Trovati {len(documenti_fatti)} articoli già processati. Riprendo da dove mi ero fermato...")

print(f"Inizio calcolo prezzi e abnormal return. (Premi Ctrl+C per fermare il processo e salvare)\n")

for index, row in df_testi.iterrows():
    chiave_univoca = row['Azienda'] + str(row['Data']) + str(row['Ora'])

    if chiave_univoca in documenti_fatti:
        continue

    try:
        ticker = mappa_ticker[row['Azienda']]
        print(f"Elaboro prezzi per {row['Azienda']} del {row['Data']} alle {row['Ora']}...")

        # Interroga Polygon (7 valori restituiti)
        p_t0, p_t30, delta_stock, delta_idx, abn_ret, vol_t0, ora_mercato = ottieni_prezzi_polygon(ticker, str(row['Data']), str(row['Ora']))

        # Crea la riga completa
        nuova_riga = pd.DataFrame({
            'Azienda': [row['Azienda']],
            'Ticker': [ticker],
            'Tipo_Documento': [row['Tipo']],
            'Data_SEC': [row['Data']],
            'Ora_SEC': [row['Ora']],
            'Ora_Effettiva_Mercato': [ora_mercato],
            'Prezzo_T0': [p_t0],
            'Prezzo_T30': [p_t30],
            'Volume_T0': [vol_t0],
            'Delta_Azienda_%': [delta_stock],
            'Delta_Mercato_%': [delta_idx],
            'Abnormal_Return_%': [abn_ret],
            'Articolo_EN': [row['Articolo']] # Rimane in inglese per l'NLP
        })

        # Salva in APPEND
        include_header = not os.path.exists(FILE_OUTPUT)
        nuova_riga.to_csv(FILE_OUTPUT, mode='a', index=False, header=include_header)

    except KeyboardInterrupt:
        print("\n[!] Interruzione manuale. Il processo si è fermato in sicurezza. Dati salvati.")
        break
    except Exception as e:
        print(f"Errore strano: {e}")
        continue

print("\nElaborazione terminata! Controlla il file:", FILE_OUTPUT)

Inizio calcolo prezzi e abnormal return. (Premi Ctrl+C per fermare il processo e salvare)

Elaboro prezzi per Microsoft del 2026-01-28 alle 21:07:34...
Elaboro prezzi per Microsoft del 2026-01-28 alle 21:04:38...
Elaboro prezzi per Microsoft del 2025-12-08 alle 21:00:35...
Elaboro prezzi per Microsoft del 2025-10-29 alle 20:10:48...
Elaboro prezzi per Microsoft del 2025-10-29 alle 20:07:42...
Elaboro prezzi per Apple del 2026-01-30 alle 11:01:32...
Elaboro prezzi per Apple del 2026-01-29 alle 21:30:33...
Elaboro prezzi per Apple del 2026-01-02 alle 21:30:52...
Elaboro prezzi per Apple del 2025-12-05 alle 21:31:42...
Elaboro prezzi per Apple del 2025-10-31 alle 10:01:26...
Elaboro prezzi per Nvidia del 2026-01-23 alle 22:01:33...
Elaboro prezzi per Nvidia del 2025-11-19 alle 21:36:17...
Elaboro prezzi per Nvidia del 2025-11-19 alle 21:20:44...
Elaboro prezzi per Nvidia del 2025-08-27 alle 20:52:07...
Elaboro prezzi per Nvidia del 2025-08-27 alle 20:22:14...

Elaborazione terminata! Cont

In [ ]:
import pandas as pd

# 1. Carichiamo il NUOVO file appena creato (versione quantitativa)
df_verifica = pd.read_csv("dataset_finale_quant.csv")

# 2. Stampiamo un riepilogo generale (quante righe, quante colonne, presenza di dati nulli)
print("=== INFO GENERALI DATASET ===")
print(df_verifica.info())
print("-" * 50)

# 3. Controlliamo se i calcoli finanziari e di Abnormal Return hanno senso
# Mostriamo le nuove colonne chiave introdotte nell'ultimo aggiornamento
print("\n=== VERIFICA METRICHE QUANTITATIVE (Prime 5 righe) ===")
colonne_finanza = ['Azienda', 'Tipo_Documento', 'Data_SEC', 'Volume_T0', 'Delta_Azienda_%', 'Delta_Mercato_%', 'Abnormal_Return_%']
print(df_verifica[colonne_finanza].head())
print("-" * 50)

# 4. Controlliamo la purezza del testo per il text mining
# Ricorda: i dataset devono essere testi/parole rigorosamente mantenuti nel loro inglese originale
print("\n=== VERIFICA TESTO (Estratto primo documento) ===")
primo_articolo = str(df_verifica['Articolo_EN'].iloc[0])
estratto_parole = ' '.join(primo_articolo.split()[:50]) # Prende le prime 50 parole
print(estratto_parole + " [...]")

=== INFO GENERALI DATASET ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Azienda                15 non-null     object 
 1   Ticker                 15 non-null     object 
 2   Tipo_Documento         15 non-null     object 
 3   Data_SEC               15 non-null     object 
 4   Ora_SEC                15 non-null     object 
 5   Ora_Effettiva_Mercato  15 non-null     object 
 6   Prezzo_T0              15 non-null     float64
 7   Prezzo_T30             15 non-null     float64
 8   Volume_T0              15 non-null     int64  
 9   Delta_Azienda_%        15 non-null     float64
 10  Delta_Mercato_%        15 non-null     object 
 11  Abnormal_Return_%      15 non-null     object 
 12  Articolo_EN            15 non-null     object 
dtypes: float64(3), int64(1), object(9)
memory usage: 1.7+ KB
None
--------------------

In [ ]:
import pandas as pd
import yfinance as yf

# 1. Caricamento del dataset esistente
# Usa il nome dell'ultimo file che hai generato con la Fase 2
NOME_FILE_INPUT = "dataset_finale_quant.csv"
NOME_FILE_OUTPUT = "dataset_quant_con_beta.csv"

print(f"Caricamento di {NOME_FILE_INPUT}...")
try:
    df = pd.read_csv(NOME_FILE_INPUT)
except FileNotFoundError:
    print(f"Errore: Il file {NOME_FILE_INPUT} non è stato trovato nella cartella.")
    raise

# Pulizia preventiva: assicuriamoci che le colonne per i calcoli siano numeriche (gestisce gli "Err Index")
df['Delta_Azienda_%'] = pd.to_numeric(df['Delta_Azienda_%'], errors='coerce')
df['Delta_Mercato_%'] = pd.to_numeric(df['Delta_Mercato_%'], errors='coerce')

# 2. Estrazione dei Ticker univoci per minimizzare le chiamate a Yahoo Finance
tickers_univoci = df['Ticker'].dropna().unique()
dizionario_beta = {}

print("\nScaricamento dei Beta da Yahoo Finance in corso...")
for ticker in tickers_univoci:
    try:
        # Crea l'oggetto Ticker e scarica il dizionario delle info
        info = yf.Ticker(ticker).info

        # Cerca la chiave 'beta'. Se per qualche motivo manca, usa 1.0 (impatto mercato 1:1)
        beta = info.get('beta', 1.0)
        dizionario_beta[ticker] = round(beta, 2)
        print(f"- {ticker}: Beta trovato = {dizionario_beta[ticker]}")
    except Exception as e:
        print(f"- Errore nel download per {ticker}: {e}. Imposto Beta di default a 1.0")
        dizionario_beta[ticker] = 1.0

# 3. Mappatura del Beta nel dataset
# Associa ad ogni riga il Beta corrispondente al suo Ticker
df['Beta_yfinance'] = df['Ticker'].map(dizionario_beta)

# 4. Calcolo del VERO Abnormal Return (Aggiustato per il Beta)
# Formula: Abnormal Return = Delta Azione - (Beta * Delta Mercato)
df['Abnormal_Return_Beta_%'] = df['Delta_Azienda_%'] - (df['Beta_yfinance'] * df['Delta_Mercato_%'])

# Arrotondiamo a due decimali
df['Abnormal_Return_Beta_%'] = df['Abnormal_Return_Beta_%'].round(2)

# Riordino estetico delle colonne: mettiamo i nuovi calcoli subito prima del testo dell'articolo
colonne = list(df.columns)
if 'Articolo_EN' in colonne:
    colonne.remove('Articolo_EN')
    if 'Beta_yfinance' in colonne: colonne.remove('Beta_yfinance')
    if 'Abnormal_Return_Beta_%' in colonne: colonne.remove('Abnormal_Return_Beta_%')

    # Riappendiamo in ordine logico
    colonne.extend(['Beta_yfinance', 'Abnormal_Return_Beta_%', 'Articolo_EN'])
    df = df[colonne]

# 5. Salvataggio del nuovo dataset
df.to_csv(NOME_FILE_OUTPUT, index=False)
print(f"\nOperazione completata! Il dataset aggiornato è stato salvato come: {NOME_FILE_OUTPUT}")

# Stampa di verifica delle sole righe dove il calcolo è andato a buon fine (ignorando i NaN)
print("\n=== VERIFICA DEL NUOVO CALCOLO (Prime 5 righe valide) ===")
colonne_verifica = ['Ticker', 'Delta_Azienda_%', 'Delta_Mercato_%', 'Beta_yfinance', 'Abnormal_Return_Beta_%']
print(df.dropna(subset=['Abnormal_Return_Beta_%'])[colonne_verifica].head())

Caricamento di dataset_finale_quant.csv...

Scaricamento dei Beta da Yahoo Finance in corso...
- MSFT: Beta trovato = 1.08
- AAPL: Beta trovato = 1.11
- NVDA: Beta trovato = 2.31

Operazione completata! Il dataset aggiornato è stato salvato come: dataset_quant_con_beta.csv

=== VERIFICA DEL NUOVO CALCOLO (Prime 5 righe valide) ===
  Ticker  Delta_Azienda_%  Delta_Mercato_%  Beta_yfinance  \
0   MSFT             0.85             0.49           1.08   
1   MSFT             3.06             0.84           1.08   
2   MSFT            -0.19             0.09           1.08   
3   MSFT             1.68             0.17           1.08   
5   AAPL             0.18             0.22           1.11   

   Abnormal_Return_Beta_%  
0                    0.32  
1                    2.15  
2                   -0.29  
3                    1.50  
5                   -0.06  


In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import re

# --- FUNZIONE DI ESTRAZIONE 8-K ---
def estrai_succo_8k(testo_grezzo):
    # Cerca l'inizio di qualsiasi Item tra 1.00 e 8.99
    pattern_inizio = re.search(r'Item\s*[1-8]\.\d{2}', testo_grezzo, re.IGNORECASE)

    # Cerca la fine: Item 9.01 oppure SIGNATURE (o SIGNATURES)
    pattern_fine = re.search(r'(Item\s*9\.01|SIGNATURES?)', testo_grezzo, re.IGNORECASE)

    if pattern_inizio:
        inizio = pattern_inizio.start()
        # Se trova la fine taglia lì, altrimenti prende tutto fino in fondo
        fine = pattern_fine.start() if pattern_fine else len(testo_grezzo)

        testo_pulito = testo_grezzo[inizio:fine].strip()
        testo_pulito = re.sub(r'\s+', ' ', testo_pulito)
        return testo_pulito

    return testo_grezzo # Se il documento è anomalo, te lo restituisce intero per sicurezza

# 1. Configurazione Aziende (Solo Microsoft)
aziende_cik = {
    'Microsoft': '0000789019'
}

# IMPORTANTE: Inserisci la tua email reale per la SEC
headers = {'User-Agent': 'TuoNome TuaEmail@esempio.com'}

lista_df = []
print("Recupero degli indici SEC per Microsoft...")

# 2. Ciclo per scaricare i metadati
for nome_azienda, cik in aziende_cik.items():
    url_submissions = f"https://data.sec.gov/submissions/CIK{cik.zfill(10)}.json"

    response = requests.get(url_submissions, headers=headers)
    if response.status_code == 200:
        data = response.json()
        filings = data['filings']['recent']

        df_temp = pd.DataFrame({
            'Azienda': nome_azienda,
            'CIK': cik,
            'Data': filings['filingDate'],
            'Ora': filings['acceptanceDateTime'],
            'Tipo': filings['form'],
            'Accession': filings['accessionNumber'],
            'PrimaryDoc': filings['primaryDocument']
        })

        # 3. Filtriamo SOLO 8-K e teniamo SOLO I PRIMI 5
        df_filtrato = df_temp[df_temp['Tipo'] == '8-K'].head(5).copy()
        lista_df.append(df_filtrato)
        print(f"- Trovati i 5 documenti 8-K per {nome_azienda}")
    else:
        print(f"Errore {response.status_code} per {nome_azienda}")

    time.sleep(0.2)

df_articoli = pd.concat(lista_df, ignore_index=True)

# 4. Funzione per scaricare, pulire l'HTML e applicare l'estrazione intelligente
def download_sec_text(cik, accession, primary_doc):
    time.sleep(0.2)

    accession_clean = accession.replace('-', '')
    cik_short = str(int(cik))
    url_doc = f"https://www.sec.gov/Archives/edgar/data/{cik_short}/{accession_clean}/{primary_doc}"

    try:
        res = requests.get(url_doc, headers=headers)
        if res.status_code == 200:
            soup = BeautifulSoup(res.content, 'html.parser')

            for tag in soup(["script", "style", "ix:header", "ix:hidden"]):
                tag.decompose()
            for hidden_tag in soup.find_all(style=lambda value: value and 'display:none' in value.replace(' ', '')):
                hidden_tag.decompose()

            testo_grezzo = soup.get_text(separator=' ', strip=True)

            # Applichiamo la funzione RegEx per estrarre solo la vera notizia in inglese
            testo_estratto = estrai_succo_8k(testo_grezzo)
            return testo_estratto
        else:
            return f"Errore HTTP {res.status_code}"
    except Exception as e:
        return f"Errore nel download: {e}"

print("\nDownload e pulizia intelligente in corso dei 5 documenti 8-K...")

# 5. Applichiamo la funzione riga per riga
df_articoli['Articolo'] = df_articoli.apply(
    lambda row: download_sec_text(row['CIK'], row['Accession'], row['PrimaryDoc']), axis=1
)

# 6. Pulizia finale e salvataggio
df_articoli['Ora'] = pd.to_datetime(df_articoli['Ora']).dt.time
df_finale = df_articoli[['Azienda', 'Tipo', 'Data', 'Ora', 'Articolo']]

print(df_finale)

# Salvataggio su file con nome specifico
df_finale.to_csv("fase1_testi_sec_microsoft_8k.csv", index=False)

Recupero degli indici SEC per Microsoft...
- Trovati i 5 documenti 8-K per Microsoft

Download e pulizia intelligente in corso dei 5 documenti 8-K...
     Azienda Tipo        Data       Ora  \
0  Microsoft  8-K  2026-01-28  21:04:38   
1  Microsoft  8-K  2025-12-08  21:00:35   
2  Microsoft  8-K  2025-10-29  20:07:42   
3  Microsoft  8-K  2025-09-30  20:37:10   
4  Microsoft  8-K  2025-07-30  20:08:53   

                                            Articolo  
0  Item 2.02. Results of Operations and Financial...  
1  Item 5.02. Departure of Directors or Certain O...  
2  Item 2.02. Results of Operations and Financial...  
3  Item 5.02. Departure of Directors or Certain O...  
4  Item 2.02. Results of Operations and Financial...  


In [4]:
# 1. Creiamo una nuova colonna che conta le parole usando la funzione split()
df_finale['Lunghezza_Parole'] = df_finale['Articolo'].apply(lambda x: len(str(x).split()))

# 2. Mostriamo un riepilogo chiaro con Data, Tipo e Lunghezza
print("\n=== LUNGHEZZA DEI 5 ARTICOLI ESTRATTI ===")
print(df_finale[['Data', 'Tipo', 'Lunghezza_Parole']])

# 3. (Opzionale) Calcoliamo anche la media
media_parole = df_finale['Lunghezza_Parole'].mean()
print(f"\nLunghezza media: {media_parole:.0f} parole per articolo.")


=== LUNGHEZZA DEI 5 ARTICOLI ESTRATTI ===
         Data Tipo  Lunghezza_Parole
0  2026-01-28  8-K               138
1  2025-12-08  8-K               699
2  2025-10-29  8-K               204
3  2025-09-30  8-K               131
4  2025-07-30  8-K               140

Lunghezza media: 262 parole per articolo.


/tmp/ipython-input-3472521200.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_finale['Lunghezza_Parole'] = df_finale['Articolo'].apply(lambda x: len(str(x).split()))


In [5]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import re
import os

# --- CONFIGURAZIONI DATASET NLP ---
MAX_ARTICOLI_PER_AZIENDA = 150
LUNGHEZZA_MINIMA_PAROLE = 50
NOME_FILE_CSV = "fase1_testi_sec_massivo.csv"

# Paniere di 30 aziende bilanciate per settore
aziende_cik = {
    # TECH (6)
    'Microsoft':    '0000789019',
    'Apple':        '0000320193',
    'Nvidia':       '0001045810',
    'Amazon':       '0001018724',
    'Alphabet':     '0001652044',
    'Meta':         '0001326801',

    # TECH MID-CAP (3) - vocabolario più vario
    'Intel':        '0000050863',
    'AMD':          '0000002488',
    'Salesforce':   '0001108524',

    # FINANZA (5)
    'JPMorgan':     '0000019617',
    'GoldmanSachs': '0000886982',
    'BankAmerica':  '0000070858',
    'Visa':         '0001403161',
    'Berkshire':    '0001067983',

    # HEALTHCARE / PHARMA (5)
    'JohnsonJohnson': '0000200406',
    'Pfizer':         '0000078003',
    'UnitedHealth':   '0000731766',
    'AbbVie':         '0001551152',
    'Merck':          '0000310158',

    # ENERGIA (3)
    'ExxonMobil':   '0000034088',
    'Chevron':      '0000093410',
    'ConocoPhillips':'0001163165',

    # CONSUMER / RETAIL (4)
    'CocaCola':     '0000021344',
    'Walmart':      '0000104169',
    'Procter':      '0000080424',
    'Nike':         '0000320187',

    # INDUSTRIALI / DIFESA (4)
    'Boeing':       '0000012927',
    'Caterpillar':  '0000018230',
    'Honeywell':    '0000773840',
    'Lockheed':     '0000936468',
}

# IMPORTANTE: Inserisci la tua email reale per la SEC
headers = {'User-Agent': 'TuoNome TuaEmail@esempio.com'}


# --- FUNZIONE DI ESTRAZIONE 8-K (Regex migliorata) ---
def estrai_succo_8k(testo_grezzo):
    # Pattern più permissivo che copre varianti tipografiche reali
    pattern_inizio = re.search(
        r'Item\s*[1-8]\s*[\.\:]\s*\d{2}', testo_grezzo, re.IGNORECASE
    )
    pattern_fine = re.search(
        r'(Item\s*9\s*[\.\:]\s*01|SIGNATURE)', testo_grezzo, re.IGNORECASE
    )

    if pattern_inizio:
        inizio = pattern_inizio.start()
        fine = pattern_fine.start() if pattern_fine else min(inizio + 15000, len(testo_grezzo))
        testo_pulito = testo_grezzo[inizio:fine].strip()
        testo_pulito = re.sub(r'\s+', ' ', testo_pulito)
        return testo_pulito

    # Fallback: primi 10.000 caratteri se non trova i marker
    return re.sub(r'\s+', ' ', testo_grezzo[:10000])


# --- FUNZIONE DOWNLOAD TESTO SEC ---
def download_sec_text(cik, accession, primary_doc):
    time.sleep(0.2)
    accession_clean = accession.replace('-', '')
    url_doc = f"https://www.sec.gov/Archives/edgar/data/{str(int(cik))}/{accession_clean}/{primary_doc}"

    try:
        res = requests.get(url_doc, headers=headers)
        if res.status_code == 200:
            soup = BeautifulSoup(res.content, 'lxml')

            for tag in soup(["script", "style", "ix:header", "ix:hidden"]):
                tag.decompose()
            for hidden_tag in soup.find_all(style=lambda value: value and 'display:none' in value.replace(' ', '')):
                hidden_tag.decompose()

            testo_grezzo = soup.get_text(separator=' ', strip=True)
            return estrai_succo_8k(testo_grezzo)
        else:
            return f"Errore HTTP {res.status_code}"
    except Exception as e:
        return f"Errore: {e}"


# --- 1. RECUPERO METADATI ---
lista_df = []
print(f"Recupero degli indici SEC per {len(aziende_cik)} aziende...")

for nome_azienda, cik in aziende_cik.items():
    url_submissions = f"https://data.sec.gov/submissions/CIK{cik.zfill(10)}.json"

    response = requests.get(url_submissions, headers=headers)
    if response.status_code == 200:
        data = response.json()
        filings = data['filings']['recent']

        df_temp = pd.DataFrame({
            'Azienda':     nome_azienda,
            'CIK':         cik,
            'Data':        filings['filingDate'],
            'Ora_UTC':     filings['acceptanceDateTime'],  # timestamp completo con timezone
            'Tipo':        filings['form'],
            'Accession':   filings['accessionNumber'],     # ID univoco — fondamentale per Polygon
            'PrimaryDoc':  filings['primaryDocument']
        })

        df_filtrato = df_temp[df_temp['Tipo'] == '8-K'].head(MAX_ARTICOLI_PER_AZIENDA).copy()
        lista_df.append(df_filtrato)
        print(f"  - {nome_azienda}: {len(df_filtrato)} documenti 8-K trovati")
    else:
        print(f"  - Errore {response.status_code} per {nome_azienda}")

    time.sleep(0.2)

df_master = pd.concat(lista_df, ignore_index=True)
totale_teorico = len(df_master)
print(f"\nTotale documenti da scaricare: {totale_teorico}")


# --- 2. SISTEMA DI RESUME ---
# Carica gli Accession già processati per non rifare il lavoro
documenti_fatti = set()
if os.path.exists(NOME_FILE_CSV):
    df_esistente = pd.read_csv(NOME_FILE_CSV)
    documenti_fatti = set(df_esistente['Accession'].tolist())
    print(f"File esistente trovato: {len(documenti_fatti)} documenti già scaricati, li salto.")
else:
    print("Nessun file esistente. Inizio da zero.")

df_da_fare = df_master[~df_master['Accession'].isin(documenti_fatti)]
totale_da_fare = len(df_da_fare)
print(f"Documenti da scaricare ora: {totale_da_fare}")
print(f"Avvio download... (mettiti comodo)\n")


# --- 3. DOWNLOAD CON APPEND PROGRESSIVO E RESUME ---
for i, (index, row) in enumerate(df_da_fare.iterrows()):
    try:
        print(f"[{i+1}/{totale_da_fare}] {row['Azienda']} — {row['Data']} ...")

        testo = download_sec_text(row['CIK'], row['Accession'], row['PrimaryDoc'])
        n_parole = len(str(testo).split())

        # Scarta documenti troppo corti
        if n_parole < LUNGHEZZA_MINIMA_PAROLE:
            print(f"  -> Scartato (solo {n_parole} parole)")
            continue

        nuova_riga = pd.DataFrame([{
            'Azienda':        row['Azienda'],
            'Tipo':           row['Tipo'],
            'Data':           row['Data'],
            'Ora_UTC':        row['Ora_UTC'],   # timestamp completo per Polygon
            'Accession':      row['Accession'], # ID univoco
            'Lunghezza_Parole': n_parole,
            'Articolo':       testo
        }])

        # Append riga per riga: sicuro in caso di crash
        include_header = not os.path.exists(NOME_FILE_CSV)
        nuova_riga.to_csv(NOME_FILE_CSV, mode='a', index=False, header=include_header)

    except KeyboardInterrupt:
        print("\n[!] Interruzione manuale. Il progresso è salvato, puoi riprendere da qui.")
        break
    except Exception as e:
        print(f"  -> Errore generico su {row['Accession']}: {e} — continuo.")
        continue


# --- 4. RIEPILOGO FINALE ---
if os.path.exists(NOME_FILE_CSV):
    df_check = pd.read_csv(NOME_FILE_CSV)
    print(f"\nDataset finale: {len(df_check)} documenti salvati in '{NOME_FILE_CSV}'")
    print("\nDistribuzione per azienda:")
    print(df_check.groupby('Azienda')['Accession'].count().sort_values(ascending=False).to_string())

Recupero degli indici SEC per 30 aziende...
  - Microsoft: 66 documenti 8-K trovati
  - Apple: 105 documenti 8-K trovati
  - Nvidia: 59 documenti 8-K trovati
  - Amazon: 60 documenti 8-K trovati
  - Alphabet: 38 documenti 8-K trovati
  - Meta: 19 documenti 8-K trovati
  - Intel: 118 documenti 8-K trovati
  - AMD: 112 documenti 8-K trovati
  - Salesforce: 21 documenti 8-K trovati
  - JPMorgan: 21 documenti 8-K trovati
  - GoldmanSachs: 14 documenti 8-K trovati
  - BankAmerica: 16 documenti 8-K trovati
  - Visa: 150 documenti 8-K trovati
  - Berkshire: 73 documenti 8-K trovati
  - JohnsonJohnson: 99 documenti 8-K trovati
  - Pfizer: 65 documenti 8-K trovati
  - UnitedHealth: 91 documenti 8-K trovati
  - AbbVie: 137 documenti 8-K trovati
  - Merck: 70 documenti 8-K trovati
  - ExxonMobil: 107 documenti 8-K trovati
  - Chevron: 101 documenti 8-K trovati
  - ConocoPhillips: 80 documenti 8-K trovati
  - CocaCola: 117 documenti 8-K trovati
  - Walmart: 48 documenti 8-K trovati
  - Procter: 86

/tmp/ipython-input-2365022125.py:94: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(res.content, 'lxml')


[2/2447] Microsoft — 2025-12-08 ...
[3/2447] Microsoft — 2025-10-29 ...
[4/2447] Microsoft — 2025-09-30 ...
[5/2447] Microsoft — 2025-07-30 ...
[6/2447] Microsoft — 2025-07-01 ...
[7/2447] Microsoft — 2025-04-30 ...
[8/2447] Microsoft — 2025-01-29 ...
[9/2447] Microsoft — 2025-01-22 ...
[10/2447] Microsoft — 2024-12-11 ...
[11/2447] Microsoft — 2024-12-03 ...
[12/2447] Microsoft — 2024-10-30 ...
[13/2447] Microsoft — 2024-08-21 ...
[14/2447] Microsoft — 2024-07-30 ...
[15/2447] Microsoft — 2024-04-25 ...
[16/2447] Microsoft — 2024-01-30 ...
[17/2447] Microsoft — 2024-01-19 ...
[18/2447] Microsoft — 2023-12-08 ...
[19/2447] Microsoft — 2023-11-06 ...
[20/2447] Microsoft — 2023-10-30 ...
[21/2447] Microsoft — 2023-10-24 ...
[22/2447] Microsoft — 2023-10-16 ...
[23/2447] Microsoft — 2023-10-13 ...
[24/2447] Microsoft — 2023-10-11 ...
[25/2447] Microsoft — 2023-07-25 ...
[26/2447] Microsoft — 2023-07-03 ...
[27/2447] Microsoft — 2023-04-25 ...
[28/2447] Microsoft — 2023-01-24 ...
[29/2447]

In [7]:
import pandas as pd

df = pd.read_csv("fase1_testi_sec_massivo.csv")

stats = df['Lunghezza_Parole'].describe(percentiles=[0.05, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99])
print(stats)
print(df.groupby('Azienda')['Lunghezza_Parole'].describe(percentiles=[0.25, 0.50, 0.75]))

count    2192.000000
mean      371.943887
std       431.140447
min        50.000000
5%         65.000000
25%       124.750000
50%       224.500000
75%       452.000000
90%       797.900000
95%      1245.500000
99%      2006.520000
max      6122.000000
Name: Lunghezza_Parole, dtype: float64
                count        mean         std    min     25%    50%      75%  \
Azienda                                                                        
AMD             112.0  541.919643  460.466531   79.0  275.75  389.5   635.50   
AbbVie          118.0  357.627119  320.582930   50.0  115.75  278.0   554.00   
Alphabet         38.0  285.736842  224.831430   53.0  155.50  206.5   335.25   
Apple           104.0  243.163462  184.010928   64.0  106.00  126.0   367.75   
BankAmerica      16.0  391.187500  260.220732   80.0  214.25  331.0   416.25   
Berkshire        72.0  290.763889  256.587601   51.0   53.00  245.5   450.00   
Boeing           76.0  419.263158  420.837938   50.0  150.75  286.0  